# NFL Data Scrape
##### This is for illustration and personal interest purposes only. Some parts of the code are slightly tweaked so that it does not run as is. Further, input functions to the below function have been removed from the notebook. No data analysis is shared as all data is property of profootballreference.com. 

In [1]:
# import libraries
import pandas as pd
import os 
import numpy as np
from bs4 import BeautifulSoup, Comment
import requests
import urllib.request, urllib.parse, urllib.error
import datetime
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import json
import timeit
import sqlite3 as sql
import time

## 2. Scraping an entire seasons worth of data

##### Below is the function which actually pulls data. It takes a list of games as input and returns 4 data frames of different types of data. Again, it will not run as is. Also, it is not completely optimized, but runtime is fine for my purposes. 

In [4]:
def full_year_scrape(games: list):
    months = {'Sep': 9,
         'Aug': 8,
         'Oct': 10,
         'Nov': 11,
         'Dec': 12,
         'Jan': 1,
         'Feb': 2}
    meta_df = pd.DataFrame(columns=['game_id', 'day', 'date', 'time', 'stadium', 'attendance', 'duration'])
    score_by_quarter_df = pd.DataFrame(columns=['game_id','a1', 'a2', 'a3', 'a4', 'aOT', 'aTot', 'h1', 'h2', 'h3', 'h4', 'hOT', 'hTot'])
    game_info_df = pd.DataFrame(columns=['game_id', 'won_toss', 'won_ot_toss', 'roof', 'surface', 'duartion', 'attendance', 'weather', 'spread', 'over_under'])
    team_stats_df = pd.DataFrame(columns=[['game_id', 'Away_Team','First Downs_Away','Rush-Yds-TDs_Away','Cmp-Att-Yd-TD-INT_Away', \
         'Sacked-Yards_Away','Net Pass Yards_Away','Total Yards_Away','Fumbles-Lost_Away','Turnovers_Away', \
        'Penalties-Yards_Away','Third Down Conv._Away','Fourth Down Conv._Away','Time of Possession_Away','Home_Team', \
         'First Downs_Home','Rush-Yds-TDs_Home','Cmp-Att-Yd-TD-INT_Home','Sacked-Yards_Home','Net Pass Yards_Home', \
         'Total Yards_Home','Fumbles-Lost_Home','Turnovers_Home','Penalties-Yards_Home','Third Down Conv._Home','Fourth Down Conv._Home','Time of Possession_Home']])
    driver = webdriver.Chrome(executable_path='C:\Program Files\Chrome\chromedriver.exe')
    for game in games:
        overtime = False
        # get this game
        driver.get(game)
        # first grab and store the metadata information
        error = 1
        while error == 1:
            try:
                meta = driver.find_element_by_class_name('scorebox_meta').find_elements_by_tag_name('div')
                error = 0
            except:
                error = 1
                print('waiting...')
                time.sleep(5)
        final_meta_data = []
        for i, element in enumerate(meta):
            if i==0:
                final_meta_data.append(element.get_attribute('innerHTML').split(' ')[0])
                date = element.get_attribute('innerHTML').split(' ')[1:]
                date_time = datetime.date(int(date[2]), int(months[date[0]]), int(date[1].replace(',','')))
                final_meta_data.append(date_time)
            if i==1 or i==4:
                quit
            if i==2 or i==3:
                final_meta_data.append(element.find_element_by_tag_name('a').get_property('innerHTML'))
        # now get the score_by_quarter breakdown
        score_by_quarter = driver.find_element_by_class_name('linescore_wrap').find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
        away_scores = [i.get_attribute('innerHTML') for i in score_by_quarter[0].find_elements_by_tag_name('td')[2:]]
        home_scores = [i.get_attribute('innerHTML') for i in score_by_quarter[1].find_elements_by_tag_name('td')[2:]]
        if len(away_scores) == 5:
            away_scores.insert(4, np.NaN)
            home_scores.insert(4, np.NaN)
        else:
            overtime = True
        scores_final = away_scores + home_scores
        # now getting game info 
        game_info = driver.find_element_by_id('all_game_info').find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
        quit
        if overtime == False:
            if len(game_info) == 7:
                for i, data in enumerate(game_info):
                    if i==4:
                        final_game_data.append(data.find_element_by_tag_name('a').get_attribute('innerHTML'))
                    elif i==6:
                        final_game_data.append(data.find_element_by_tag_name('tdds').get_attribute('innerHTML').split(' ')[0])
                    else:
                        final_game_data.append(data.find_element_by_tag_name('tdas').get_attribute('innerHTML'))
                final_game_data.insert(1, np.NaN)
                final_game_data.insert(6, np.NaN)
            else:
                for i, data in enumerate(game_info):
                    if i==4:
                        final_game_data.append(data.find_element_by_tag_name('ad').get_attribute('innerHTML'))
                    elif i==7:
                        final_game_data.append(data.find_element_by_tag_name('pa').get_attribute('innerHTML').split(' ')[0])
                    else:
                        final_game_data.append(data.find_element_by_tag_name('td').get_attribute('innerHTML'))
                final_game_data.insert(1, np.NaN)
        else:
            if len(game_info) == 8:
                for i, data in enumerate(game_info):
                    if i==14:
                        final_game_data.append(data.find_element_by_tag_name('a').get_attribute('innerHTML'))
                    elif i==7:
                        final_game_data.append(data.find_element_by_tag_name('ja').get_attribute('innerHTML').split(' ')[0])
                    else:
                        final_game_data.append(data.find_element_by_tag_name('td').get_attribute('innerHTML'))
                final_game_data.insert(1, np.NaN)
            else:
                for i, data in enumerate(game_info):
                    if i==5:
                        final_game_data.append(data.find_element_by_tag_name('a').get_attribute('innerHTML'))
                    elif i==8:
                        final_game_data.append(data.find_element_by_tag_name('ts').get_attribute('innerHTML').split(' ')[0])
                    else:
                        final_game_data.append(data.find_element_by_tag_name('td').get_attribute('innerHTML'))
        if len(final_game_data) == 10:
            final_game_data.pop(9)
        # into team stats
        team_stats = driver.find_element_by_id('team_stats')
        teams = [i.get_attribute('innerHTML') for i in team_stats.find_element_by_tag_name('thead').find_elements_by_tag_name('th')]
        away_team, home_team = teams[1], teams[2]
        rows = team_stats.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
        for row in rows:
            away_stats.append(row.find_elements_by_tag_name('td')[0].get_attribute('innerHTML'))
            home_stats.append(row.find_elements_by_tag_name('td')[1].get_attribute('innerHTML'))
        away_stats.insert(0, away_team)
        home_stats.insert(0, home_team)
        final_team_stats = away_stats + home_stats
        # all data is ready, just need to add a gameID to each row before
        game_id = away_team + home_team + str(date_time.year) + str(date_time.month).zfill(2) + str(date_time.day).zfill(2)
        for l in [final_team_stats, scores_final, final_meta_data, final_game_data]:
            l.insert(0, game_id)
        team_stats_df.loc[len(team_stats_df)] = final_team_stats
        score_by_quarter_df.loc[len(score_by_quarter_df)] = scores_final
        meta_df.loc[len(meta_df)] = final_meta_data
        game_info_df.loc[len(game_info_df)] = final_game_data
    driver.close()
    
    
    return meta_df, score_by_quarter_df, game_info_df, team_stats_df

In [5]:
test = full_year_scrape(games)

waiting...


##### Below is an example of one dataframe that is produced for reference. 

In [11]:
test[3]

,game_id,Away_Team,First Downs_Away,Rush-Yds-TDs_Away,Cmp-Att-Yd-TD-INT_Away,Sacked-Yards_Away,Net Pass Yards_Away,Total Yards_Away,Fumbles-Lost_Away,Turnovers_Away,...,Cmp-Att-Yd-TD-INT_Home,Sacked-Yards_Home,Net Pass Yards_Home,Total Yards_Home,Fumbles-Lost_Home,Turnovers_Home,Penalties-Yards_Home,Third Down Conv._Home,Fourth Down Conv._Home,Time of Possession_Home
0,CHIMIN20000903,CHI,23,25-153-1,27-41-290-2-0,2-18,272,425,2-1,1,...,13-23-190-0-1,1-2,188,374,2-0,1,4-20,6-13,0-0,27:26
1,INDKAN20000903,IND,20,32-119-1,22-32-273-1-1,1-6,267,386,1-1,2,...,16-37-212-2-1,1-6,206,280,0-0,1,6-50,5-15,0-1,26:47
2,CARWAS20000903,CAR,12,20-112-0,17-26-183-1-0,6-59,124,236,1-1,1,...,25-36-234-0-0,0-0,234,396,0-0,0,7-64,5-12,0-1,33:14
3,JAXCLE20000903,JAX,28,40-119-2,24-34-301-1-0,4-22,279,398,1-0,0,...,19-27-160-1-0,1-7,153,249,1-1,1,3-25,3-10,1-2,22:57
4,SFOATL20000903,SFO,23,24-92-1,23-36-253-3-1,1-6,247,339,3-0,1,...,16-31-264-2-0,0-0,264,359,2-1,1,4-51,4-11,0-0,31:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,BALTEN20010107,BAL,6,23-49-1,5-17-117-0-0,3-32,85,134,0-0,0,...,25-47-195-0-1,1-4,191,317,0-0,1,6-55,5-16,0-2,40:29
255,PHINYG20010107,PHI,11,14-46-0,20-41-181-1-1,6-41,140,186,3-2,3,...,12-19-125-0-0,1-0,125,237,3-3,3,4-18,2-13,0-0,36:09
256,MINNYG20010114,MIN,9,9-54-0,13-28-78-0-3,4-18,60,114,2-2,5,...,29-40-385-5-2,1-5,380,518,1-0,2,4-36,8-16,1-1,42:22
257,BALOAK20010114,BAL,12,46-110-0,9-18-190-1-1,2-18,172,282,2-1,2,...,19-37-187-0-4,4-20,167,191,2-1,5,5-36,2-14,0-0,25:22
